<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## Non-maximum suppression

First the commonly used NMS with bounding boxes, that prioritizes either confidence score (default) or bounding box area.

In [1]:
#|output: asis
#| echo: false
show_doc(non_max_suppression_fast)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L18){target="_blank" style="float:right; font-size:smaller"}

### non_max_suppression_fast

>      non_max_suppression_fast (boxes, scores, overlap_thresh:float,
>                                sort_criterion:str='score')

Possibility to sort boxes by score (default) or area

Non-max suppression can in theory be applied also on polygons, but it hasn't been used in any publications as far as I know.

If [`non_max_suppression_poly`](https://mayrajeo.github.io/drone_detector/processing.postprocessing.html#non_max_suppression_poly) is used to eliminate polygons, threshold might need to be smaller than typical value of 0.7 that is used.

In [2]:
#|output: asis
#| echo: false
show_doc(non_max_suppression_poly)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L88){target="_blank" style="float:right; font-size:smaller"}

### non_max_suppression_poly

>      non_max_suppression_poly (geoms, scores, overlap_thresh:float,
>                                sort_criterion:str='score')

Possibility to sort geoms by score (default) or area

Some utils to run above functions to `GeoDataFrames`

In [3]:
#|output: asis
#| echo: false
show_doc(do_min_rot_rectangle_nms)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L147){target="_blank" style="float:right; font-size:smaller"}

### do_min_rot_rectangle_nms

>      do_min_rot_rectangle_nms (gdf:geopandas.geodataframe.GeoDataFrame,
>                                nms_thresh=0.7, crit='score')

In [4]:
#|output: asis
#| echo: false
show_doc(do_poly_nms)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L140){target="_blank" style="float:right; font-size:smaller"}

### do_poly_nms

>      do_poly_nms (gdf:geopandas.geodataframe.GeoDataFrame, nms_thresh=0.1,
>                   crit='score')

In [5]:
#|output: asis
#| echo: false
show_doc(do_nms)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L132){target="_blank" style="float:right; font-size:smaller"}

### do_nms

>      do_nms (gdf:geopandas.geodataframe.GeoDataFrame, nms_thresh=0.7,
>              crit='score')

# Weighted boxes fusion

Originally presented by [Solovyev et al (2021)](https://arxiv.org/abs/1910.13302), and available in [https://github.com/ZFTurbo/Weighted-Boxes-Fusion]. Code presented here is modified to keep track of original bounding boxes for mask fusion, and due to numpy version requirements we do not use numba here.

As WBF expects normalized coordinates, first some helpers to normalize and denormalize geocoordinates. 

In [6]:
#|output: asis
#| echo: false
show_doc(denormalize_bbox_coords)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L170){target="_blank" style="float:right; font-size:smaller"}

### denormalize_bbox_coords

>      denormalize_bbox_coords (tot_bounds, bboxes)

In [7]:
#|output: asis
#| echo: false
show_doc(normalize_bbox_coords)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L156){target="_blank" style="float:right; font-size:smaller"}

### normalize_bbox_coords

>      normalize_bbox_coords (tot_bounds, bboxes)

In [8]:
#|output: asis
#| echo: false
show_doc(weighted_boxes_fusion)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L341){target="_blank" style="float:right; font-size:smaller"}

### weighted_boxes_fusion

>      weighted_boxes_fusion (boxes_list, scores_list, labels_list,
>                             weights=None, iou_thr=0.55, skip_box_thr=0.0,
>                             conf_type='avg', allows_overflow=False)

:param boxes_list: list of boxes predictions from each model, each box is 4 numbers.
It has 3 dimensions (models_number, model_preds, 4)
Order of boxes: x1, y1, x2, y2. We expect float normalized coordinates [0; 1]
:param scores_list: list of scores for each model
:param labels_list: list of labels for each model
:param weights: list of weights for each model. Default: None, which means weight == 1 for each model
:param iou_thr: IoU value for boxes to be a match
:param skip_box_thr: exclude boxes with score lower than this variable
:param conf_type: how to calculate confidence in weighted boxes. 'avg': average value, 'max': maximum value, 'box_and_model_avg': box and model wise hybrid weighted average, 'absent_model_aware_avg': weighted average that takes into account the absent model.
:param allows_overflow: false if we want confidence score not exceed 1.0
:return: boxes: boxes coordinates (Order of boxes: x1, y1, x2, y2).
:return: scores: confidence scores
:return: labels: boxes labels
:return: originals: original boxes

In [9]:
#|output: asis
#| echo: false
show_doc(find_matching_box_quickly)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L300){target="_blank" style="float:right; font-size:smaller"}

### find_matching_box_quickly

>      find_matching_box_quickly (boxes_list, new_box, match_iou)

Reimplementation of find_matching_box with numpy instead of loops. Gives significant speed up for larger arrays
(~100x). This was previously the bottleneck since the function is called for every entry in the array.

In [10]:
#|output: asis
#| echo: false
show_doc(get_weighted_box)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L270){target="_blank" style="float:right; font-size:smaller"}

### get_weighted_box

>      get_weighted_box (boxes, conf_type='avg')

Create weighted box for set of boxes
:param boxes: set of boxes to fuse
:param conf_type: type of confidence one of 'avg' or 'max'
:return: weighted box (label, score, weight, x1, y1, x2, y2)

In [11]:
#|output: asis
#| echo: false
show_doc(prefilter_boxes)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L208){target="_blank" style="float:right; font-size:smaller"}

### prefilter_boxes

>      prefilter_boxes (boxes, scores, labels, weights, thr)

In [12]:
#|output: asis
#| echo: false
show_doc(bb_intersection_over_union)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L188){target="_blank" style="float:right; font-size:smaller"}

### bb_intersection_over_union

>      bb_intersection_over_union (A, B)

In [13]:
#|output: asis
#| echo: false
show_doc(do_wbf)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L429){target="_blank" style="float:right; font-size:smaller"}

### do_wbf

>      do_wbf (gdf:geopandas.geodataframe.GeoDataFrame, iou_thr=0.55,
>              skip_box_thr=0.5)

Run weighted_boxes_fusion and returns a gpd.GeoDataFrame where geometries are replaced by new bounding boxes.
Do not use with instance segmentation data unless you want to replace your results with bounding boxes

In [14]:
#|output: asis
#| echo: false
show_doc(do_wsf)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L449){target="_blank" style="float:right; font-size:smaller"}

### do_wsf

>      do_wsf (gdf:geopandas.geodataframe.GeoDataFrame, iou_thr=0.55,
>              skip_box_thr=0.5)

## Smoothing and filling holes

Below functions are run before converting IceVision preds to COCO or shapefile format.

In [15]:
#|output: asis
#| echo: false
show_doc(dilate_erode)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L495){target="_blank" style="float:right; font-size:smaller"}

### dilate_erode

>      dilate_erode (preds:list)

Run dilation followed by erosion in order to smooth masks

In [16]:
#|output: asis
#| echo: false
show_doc(fill_holes)

---

[source](https://github.com/mayrajeo/drone_detector/tree/master/blob/master/drone_detector/processing/postproc.py#L487){target="_blank" style="float:right; font-size:smaller"}

### fill_holes

>      fill_holes (preds:list)

Run `binary_fill_holes` to predicted binary masks